The script `ARIMA_Model.R` starts with pre-processing the data. There was a problem in the code that I fixed based on a guess. We'll continue assuming that the fix was correct. 

First we store the data of the last two days of train data in `demand_list_ar`:

In [ ]:
n_tr = length(train)
n_ts = length(test)
demand_list_ar = lapply(1:2, function(i) train[[n_tr-2+i]])

Then we change these matrices into a long list of consecutive counts. We also do the same for the test data, and concatenate the two lists together. If we assume that test events have happened right after the train events, `vec_arima` is a log of counts of consecutive events starting from two weeks before the starting date of test events.

![](./images/cells_train_test.png)

In [ ]:
demand_ar=c(sapply(1:2, function(i) c(t(demand_list_ar[[i]]))))
demand_a=c(sapply(1:n_ts, function(x) c(t(test[[i]]))))
vec_arima=c(demand_ar,demand_a)

The train/test operation is repeated for each day in the test. Two weeks of data before the test date is concatenated to te first period of the test day. Depending on where the test day is located, the previous two weeks can contain data from train, test, or both datasets.

![](images/cells_traindata.png)

The first two weeks is sent to `calculaModelArima` function to train an ARIMA model. 

In [ ]:
train_start = 48 * (target_day-1)
twoweeks = 48 * 14
datatrain = vec_arima[(train_start+1):(train_start+twoweeks)]

# send this data to train the ARIMA model
o = calculaModelArima(datatrain)

Let's assume that order `o` is computed and jump to the prediction step. This is how prediction for one period is performed: 
- take the series containing 24\*7 periods before the target
- that series together with `o` is fed into function `predict`

![](./images/pred_slide.png)

This is repeated until all 48 periods in the target day are predicted

![](./images/pred_slide_2.png)

In [ ]:
pred_oneday = function (x) {
    
    pred_start = x + 48*(target_day-1)
    datapred = vec_arima[pred_start:(pred_start+twoweeks-1)]
    
    r <- predict(arima(datapred, order=o), n.ahead =1)$pred
    return (r)
}

predictions = sapply(1:48, function(x) pred_oneday(x))